In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import datetime
import os
import random

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Parameters
toy = False
run_all = True
path = '/home/ubuntu/fmriprep/'
output_dir = '../output/'
input_dir = '/home/ubuntu/brain_age/drsulliv3/'

n_jobs = 20 #amount of cores
cv=4
description = 'CNP_func_gridsearch_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [3]:
log_file

'CNP_func_gridsearch_age_18-08-10-02-50-17'

In [4]:
df = pd.read_csv('participants.tsv', sep='\t')
df = df[df.participant_id != 'sub-70002']
df = df[df.participant_id != 'sub-70035']
df = df[df.participant_id != 'sub-70036']

df_adhd = df[df.diagnosis=='ADHD']
subj_adhd = list(df_adhd.participant_id) 
y_adhd = list(df_adhd.age)
subj_adhd = [n[4:] for n in subj_adhd]

df_schz = df[df.diagnosis=='SCHZ']
subj_schz = list(df_schz.participant_id) 
y_schz = list(df_schz.age)
subj_schz = [n[4:] for n in subj_schz]

df_bipolar = df[df.diagnosis=='BIPOLAR']
subj_bipolar = list(df_bipolar.participant_id) 
y_bipolar = list(df_bipolar.age)
subj_bipolar = [n[4:] for n in subj_bipolar]

df_all = df[df.diagnosis !='CONTROL']
subj_all = list(df_all.participant_id) 
y_all = list(df_all.age)
subj_all = [n[4:] for n in subj_all]
df_adhd

,participant_id,diagnosis,age,gender,bart,bht,dwi,pamenc,pamret,rest,scap,stopsignal,T1w,taskswitch,ScannerSerialNumber,ghost_NoGhost
229,sub-70001,ADHD,50,F,1.0,NaN,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost
231,sub-70004,ADHD,26,F,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost
232,sub-70007,ADHD,32,M,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,35343.0,ghost
233,sub-70010,ADHD,48,M,1.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,1.0,1.0,35343.0,No_ghost
234,sub-70015,ADHD,50,F,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost
235,sub-70017,ADHD,24,M,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost
236,sub-70020,ADHD,50,M,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,35343.0,ghost
237,sub-70021,ADHD,33,M,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost
238,sub-70022,ADHD,21,M,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost
239,sub-70026,ADHD,37,F,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,35343.0,No_ghost


In [5]:
# # Load subject ids
# if run_all:
#     files = os.listdir(path)
#     subj = [n[4:9] for n in files if n.startswith('sub')] #here we will filter the file names to leave only subj number as bellow

#     file = 'subjlist_wfiles_CNP.txt'
#     with open(input_dir+file, 'r') as f:
#         lines = f.readlines()

# #np.load(/home/ubuntu/brain_age/drsulliv3/)
# # This was done once before on the T1. This order was randomized. 



# healthy =  ['10672', '10940', '10361', '10719', '11112', '10356', '11106',
#               '11030', '10340', '10570', '10329', '10871', '10746', '10325',
#               '10987', '10724', '11105', '10492', '10429', '10565', '10977',
#               '10316', '10460', '10678', '10704', '11143', '11067', '10274',
#               '10304', '10696', '11098', '10998', '10455', '10629', '10963',
#               '10345', '11019', '10271', '11108', '10225', '11149', '11050',
#               '10448', '10680', '11068', '11104', '10388', '10530', '10206',
#               '10692', '10438', '10228', '10762', '10708', '10235', '10788',
#               '10527', '10631', '11044', '10844', '10517', '11090', '10377',
#               '10934', '10575', '10471', '10159', '10189', '10779', '10949',
#               '11052', '11059', '10280', '11156', '11077', '10975', '10290',
#               '10557', '11066', '10686', '10891', '10525', '11061', '10321',
#               '10697', '10668', '10171', '11122', '10217', '10785', '10339',
#               '10523', '10506', '10347', '10707', '11062', '10958', '10376',
#               '10893', '10440', '10968', '10365', '10292', '11142', '10478',
#               '11088', '10912', '10855', '10487', '10273', '11097', '10638',
#               '11131', '10227', '10269', '10882', '10877', '10524', '10624',
#               '10249', '11128', '10674']


In [6]:
#len(subj)

In [7]:
# # Load ages
# df = pd.read_csv('participants.tsv', sep='\t')

# # take age from subj id
# y_age = []
# for i in subj:
#     a = df.loc[df['participant_id'] == 'sub-'+i] #whole row
#     y_age.append(int(a.age))


In [8]:
len(subj_all)

139

In [9]:
# Load volumes for each subject into a dictionary    
def filenames(subj):
    filenames = []
    for i in range(len(subj)):
        filenames.append(path+'sub-'+subj[i]+'/func/sub-'+subj[i]+'_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')
    return filenames

filenames_adhd  = filenames(subj_adhd)
filenames_schz  = filenames(subj_schz)
filenames_bipolar  = filenames(subj_bipolar)
filenames_all  = filenames(subj_all)


In [10]:
from nilearn import datasets

parcellations = datasets.fetch_atlas_basc_multiscale_2015(version='sym')

# We show here networks of 64, 197, 444
# networks_64 = parcellations['scale064']
networks_197 = parcellations['scale197']
networks_444 = parcellations['scale444']

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [11]:
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure, sym_matrix_to_vec

def func_2_corr_matrix(parcellation, name, filenames, y, subj):
    masker = NiftiLabelsMasker(labels_img=parcellation, standardize=True,
                               memory='nilearn_cache', verbose=0)
    #n_regions = 197
    
    correlations = []
    upper_triang_corrs = []
    # Initializing ConnectivityMeasure object with kind='correlation'
    connectome_measure = ConnectivityMeasure(kind='correlation')
    for filename in filenames:
        timeseries_each_subject = masker.fit_transform(filename)
        # call fit_transform from ConnectivityMeasure object
        correlation = connectome_measure.fit_transform([timeseries_each_subject])
        # saving each subject correlation to correlations
        correlations.append(correlation)
        # get the upper triangular correlation matrix of correlation
        upper_triang_corr = sym_matrix_to_vec(correlation, discard_diagonal=True)
        # saving each subject upper_triang_corr to upper_triang_corrs
        upper_triang_corrs.append(upper_triang_corr) 
    # Random     
    c = list(zip(upper_triang_corrs, y, subj))
    random.shuffle(c)
    upper_triang_corrs, y, subj = zip(*c)
    #   Split train test sets and save
    split = int(len(upper_triang_corrs)/2.)
    X_train = upper_triang_corrs[:split]
    X_test = upper_triang_corrs[split:]
    y_train = y[:split]
    y_test = y[split:]
    random_subj_train = subj[:split]
    random_subj_test = subj[split:]
    np.savez_compressed(output_dir+log_file+'CNP_func'+name+'_train_set',a=X_train, b=y_train, c=random_subj_train)
    np.savez_compressed(output_dir+log_file+'CNP_func'+name+'_test_set',a=X_test, b=y_test, c=random_subj_test)
    return correlations, upper_triang_corrs

correlations197, upper_triang_corrs197 = func_2_corr_matrix(networks_197, 'adhd_197', filenames_adhd, y_adhd, subj_adhd)
correlations444, upper_triang_corrs444 = func_2_corr_matrix(networks_444, 'adhd_444', filenames_adhd, y_adhd, subj_adhd)

correlations197, upper_triang_corrs197 = func_2_corr_matrix(networks_197, 'schz_197', filenames_schz, y_schz, subj_schz)
correlations444, upper_triang_corrs444 = func_2_corr_matrix(networks_444, 'schz_444', filenames_schz, y_schz, subj_schz)

correlations197, upper_triang_corrs197 = func_2_corr_matrix(networks_197, 'bipolar_197', filenames_bipolar, y_bipolar, subj_bipolar)
correlations444, upper_triang_corrs444 = func_2_corr_matrix(networks_444, 'bipolar_444', filenames_bipolar, y_bipolar, subj_bipolar)

# correlations197, upper_triang_corrs197 = func_2_corr_matrix(networks_197, '197', filenames_all, y_all, subj_all)
# correlations444, upper_triang_corrs444 = func_2_corr_matrix(networks_444, '444', filenames_all, y_all, subj_all)


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [12]:
# loaded = np.load(output_dir+'train_set.npz')
# X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']